In [13]:
# import torch
# from cspdarknet53 import _BuildCSPDarknet53

# # Lokasi berkas weight
# path = 'F:/DEV PY/Yolo/Knowledge-distillation-withYolov4-/yolov4.weights'

# # Panggil fungsi untuk membuat model dan mendapatkan model dan nilai terkait
# model_tuple = _BuildCSPDarknet53(weight_path=path, resume=True)

# # Dapatkan model dari elemen pertama tupel
# model = model_tuple[0]

# # Muat weights ke model
# if torch.cuda.is_available():
#     # Jika GPU tersedia, muat weights ke dalam model dan pindahkan model ke GPU
#     model.load_state_dict(torch.load(path))
#     model.to(torch.device("cuda"))
# else:
#     # Jika tidak, muat weights ke dalam model dalam mode CPU
#     model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'torch'

In [15]:
# Below we are preprocessing data for CIFAR-10. We use an arbitrary batch size of 128.
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading the CIFAR-10 dataset:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)

100%|██████████| 170498071/170498071 [00:42<00:00, 3999489.03it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [16]:
#Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

In [18]:
from cspdarknet53 import _BuildCSPDarknet53

path = 'F:/DEV PY/Yolo/Knowledge-distillation-withYolov4-/yolov4.weights'

class CSPDarknet53Model(nn.Module):
    def __init__(self, num_classes=10):
        super(CSPDarknet53Model, self).__init__()
        # Panggil fungsi untuk membangun model CSPDarknet53
        self.model, _ = _BuildCSPDarknet53(weight_path='path', resume=True)
        
        # Ubah lapisan klasifikasi terakhir sesuai dengan jumlah kelas yang diinginkan
        # Misalnya, kita ubah menjadi 10 kelas
        num_features = self.model.feature_channels[-1]  # Dapatkan jumlah fitur dari lapisan fitur terakhir
        self.model.num_classes = num_classes
        self.model.head = nn.Sequential(
            nn.Conv2d(num_features, num_classes, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
        )

    def forward(self, x):
        # Gunakan model CSPDarknet53 yang sudah dibangun
        return self.model(x)


In [19]:
class Students(nn.Module):
    def __init__(self, num_classes=10):
        super(Students, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [20]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs: A collection of batch_size images
            # labels: A vector of dimensionality batch_size with integers denoting class of each image
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # outputs: Output of the network for the collection of images. A tensor of dimensionality batch_size x num_classes
            # labels: The actual labels of the images. Vector of dimensionality batch_size
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def test(model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [21]:
torch.manual_seed(42)
nn_deep = CSPDarknet53Model(num_classes=10).to(device)
train(nn_deep, train_loader, epochs=10, learning_rate=0.001, device=device)
test_accuracy_deep = test(nn_deep, test_loader, device)

# Instantiate the lightweight network:
torch.manual_seed(42)
nn_light = Students(num_classes=10).to(device)

******************** Initing CSPDarknet53 weights ********************
initing Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
initing BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
initing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
initing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
initing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
initing BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
initing BatchNorm2d(64

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not list

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from cspdarknet53 import _BuildCSPDarknet53


path = 'F:/DEV PY/Yolo/Knowledge-distillation-withYolov4-/yolov4.weights'
model_tuple = _BuildCSPDarknet53(weight_path=path, resume=True)

# Inisialisasi model guru
model = model_tuple[0]

# Tentukan fungsi kerugian dan algoritma optimasi
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Tentukan jumlah epoch (iterasi melalui dataset)
num_epochs = 10

# Proses pelatihan
for epoch in range(num_epochs):
    model.train()  # Atur model ke mode pelatihan
    running_loss = 0.0
    for images, labels in train_loader:
        # Reset gradien parameter
        optimizer.zero_grad()

        # Lakukan langkah maju (forward pass)
        outputs = model(images)

        # Hitung loss
        loss = criterion(outputs, labels)

        # Langkah mundur (backward pass) dan optimasi
        loss.backward()
        optimizer.step()

        # Hitung total loss pada batch ini
        running_loss += loss.item() * images.size(0)

    # Cetak rata-rata loss pada epoch ini
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print('Pelatihan selesai.')

******************** Initing CSPDarknet53 weights ********************
initing Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
initing BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
initing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
initing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
initing BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
initing BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
initing Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
initing BatchNorm2d(64

RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[64, 1, 28, 28] to have 3 channels, but got 1 channels instead

In [ ]:
# Set model ke mode evaluasi
model.eval()